# Logistic Regresion with Pytorch

**Steps**
1. Design model (input, output size, forward pass)
2. Construct loss and optimizer
3. Training loop
   - Forward pass: compute predictions
   - backward pass: gradients
   - update weights 

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Data Preparing

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [3]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,stratify=y ,random_state=84)

In [5]:
#scale features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [7]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

## 2. Model

In [8]:
# wx=b, sigmoid at the end
input_size = n_features
output_size = 1
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

## 3. loss and optimizer

In [9]:
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 4. training loop

In [10]:
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()

    # update weights
    optimizer.step()

    # empty gradients
    optimizer.zero_grad()

    if (epoch +1) % 10 ==0:
        print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss: 0.6180
epoch: 20, loss: 0.4887
epoch: 30, loss: 0.4121
epoch: 40, loss: 0.3631
epoch: 50, loss: 0.3289
epoch: 60, loss: 0.3035
epoch: 70, loss: 0.2837
epoch: 80, loss: 0.2677
epoch: 90, loss: 0.2544
epoch: 100, loss: 0.2431
accuracy = 0.9649
